<a href="https://colab.research.google.com/github/sanozzz/MoneyPulseAI/blob/main/ANG_TopGainers_(v2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import logging
from datetime import datetime
from pytz import timezone
import aiohttp
import asyncio
import nest_asyncio
import json
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import requests
from textwrap import fill
from google.colab import userdata
from openai import OpenAI
import re
import pytz
import openai
import time


# Allow nested event loops
nest_asyncio.apply()

# Set up the logger globally


def setup_logger():
    """Sets up a logger with both file and console handlers, formatted with IST timezone."""
    logger = logging.getLogger("TopGainersLogger")

    # Avoid adding duplicate handlers
    if logger.hasHandlers():
        logger.handlers.clear()

    # Set the logging level
    logger.setLevel(logging.INFO)

    # Define a custom formatter with IST timezone
    class ISTFormatter(logging.Formatter):
        def formatTime(self, record, datefmt=None):
            ist = timezone("Asia/Kolkata")
            record_time = datetime.fromtimestamp(record.created, ist)
            return record_time.strftime(datefmt or "%Y-%m-%d %H:%M:%S")

    # Create the formatter
    formatter = ISTFormatter("%(asctime)s - %(levelname)s - %(message)s")

    # File handler setup
    try:
        log_file = f"/content/TopGainers_{datetime.now(timezone('Asia/Kolkata')).strftime('%Y-%m-%d')}.log"
        file_handler = logging.FileHandler(log_file, mode="a")  # Append mode
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
        print(f"File logging initialized at: {log_file}")
    except Exception as e:
        print(f"Failed to set up file handler: {e}")

    # Stream handler setup (console logging)
    try:
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)
        print("Console logging initialized.")
    except Exception as e:
        print(f"Failed to set up console handler: {e}")

    # Test the logger
    logger.info("Logger initialized successfully with IST timezone.")
    return logger


# Initialize the logger
logger = setup_logger()



def get_ist_timestamp():
    """Returns the current timestamp in IST."""
    ist = pytz.timezone('Asia/Kolkata')
    return datetime.now(ist).strftime('%Y-%m-%d %H:%M:%S')


class TopGainers:
    def __init__(self, input_path='/content/TopGainers.csv', output_path='/content/TopGainers_with_DetailedSummaries.csv', plot_path='/content/TopGainers.png'):
        """
        Initialize the TopGainers class with file paths and API keys for OpenAI and Perplexity.
        """
        self.input_path = input_path
        self.output_path = output_path
        self.plot_path = plot_path

        # Fetch API keys from Colab's userdata or environment variables
        openai_api_key = userdata.get("OPENAI_API_KEY") or os.environ.get("OPENAI_API_KEY")
        perplexity_api_key = userdata.get("PERPLEXITY_API_KEY") or os.environ.get("PERPLEXITY_API_KEY")

        if not openai_api_key:
            logger.error("OpenAI API key is not set. Please configure it in userdata or environment variables.")
            raise ValueError("OpenAI API key is missing.")
        if not perplexity_api_key:
            logger.error("Perplexity API key is not set. Please configure it in userdata or environment variables.")
            raise ValueError("Perplexity API key is missing.")

        # Initialize OpenAI client
        self.openai_api_key = openai_api_key
        self.initialize_openai_client()

        # Store the Perplexity API key for later use
        self.perplexity_api_key = perplexity_api_key
        self.api_url = "https://api.perplexity.ai/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.perplexity_api_key}",
            "Content-Type": "application/json"
        }


        # Log successful initialization
        logger.info("TopGainers class initialized successfully.")

        # Validate the input file path
        self.validate_input_path()

    def initialize_openai_client(self):
        """Initialize the OpenAI client with the API key."""
        openai.api_key = self.openai_api_key
        self.client = openai
        logger.info("OpenAI client initialized successfully.")

    def validate_input_path(self):
        """Validate the existence of the input file."""
        if not os.path.exists(self.input_path):
            logger.error(f"Input file not found at: {self.input_path}")
            raise FileNotFoundError(f"Input file not found at: {self.input_path}")
        logger.info(f"Input file located at: {self.input_path}")


    def generate_contextual_summary(self, row, max_retries=3, retry_delay=2):
        """
        Generate a conversational summary for a given stock row using Perplexity's API.
        Retries the request in case of an error.
        """
        # Determine stellar performance metrics
        stellar_quarterly_performance = (
            f"The stock has also shown stellar quarterly returns of {row['Qtr Change %']}%, "
            f"outperforming benchmarks."
            if row.get('Qtr Change %', 0) > 15 else ""
        )

        stellar_relative_performance = (
            f"The stock also delivered stellar relative returns compared to Nifty500 this month ({row['Relative returns vs Nifty500 month%']}%) "
            f"and this quarter ({row['Relative returns vs Nifty500 quarter%']}%)."
            if row.get('Relative returns vs Nifty500 month%', 0) > 15
            and row.get('Relative returns vs Nifty500 quarter%', 0) > 15
            else ""
        )

        insider_trading_commentary = (
            f"Additionally, there have been significant insider trading and SAST buys last quarter, "
            f"with more than 100,000 shares traded, indicating strong confidence in the stock's potential."
            if row.get('Insider & SAST Buys Last Quarter', 0) > 100000 else ""
        )

        ath_commentary = (
            f"The stock hit a new all-time high today at ₹{row['Day High']}, breaking the previous 52-week high last recorded on {row['Date of 52W High']}."
            if row.get('NewATH', False) else ""
        )

        # Combine all notable performance metrics
        notable_performance = " ".join(filter(None, [
            stellar_quarterly_performance,
            stellar_relative_performance,
            insider_trading_commentary,
            ath_commentary
        ]))

        # Construct the query
        query = (
            f"Generate a summary for the following stock data:\n\n"
            f"- Stock: {row.get('Stock', 'Unknown')}\n"
            f"- Day change: {row.get('Day change %', 0):.1f}%\n"
            f"- Closing price: ₹{row.get('Current Price', 'N/A')}\n"
            f"- Daily turnover: ₹{row.get('DailyTurnInCr', 'N/A')} Cr\n"
            f"- Trading volume multiple of the week: {row.get('Day volume multiple of week', 'N/A')}\n"
            f"- Percent of float traded on the primary exchange: {row.get('PercentFloatTradedPrimaryExchange', 0):.1f}%\n"
            f"- Revenue QoQ Growth: {row.get('Revenue QoQ Growth %', 0):.1f}%\n"
            f"- Basic EPS QoQ Growth: {row.get('Basic EPS QoQ Growth %', 0):.1f}%\n"
            f"- Day high: ₹{row.get('Day High', 'N/A')}\n"
            f"- 10-year high: ₹{row.get('10Yr High', 'N/A')}\n"
            f"- New All-Time High: {'Yes' if row.get('NewATH', False) else 'No'}\n"
            f"- Date of previous 52-week high: {row.get('Date of 52W High', 'N/A')}\n"
            f"- Piotroski Score: {row.get('Piotroski Score', 'N/A')}\n\n"
            f"{notable_performance}\n\n"
            f"Generate a friendly and engaging summary focusing on today's performance, recent growth, and notable highs.\n\n"
            f"Remove any text containing '**' or markdown-style formatting."
        )

        # Retry mechanism
        for attempt in range(max_retries):
            try:
                # Construct the payload for Perplexity's API
                payload = {
                    "model": "llama-3.1-sonar-huge-128k-online",
                    "messages": [{"role": "user", "content": query}]
                }

                # Make the API call
                response = requests.post(
                    "https://api.perplexity.ai/chat/completions",
                    json=payload,
                    headers={"Authorization": f"Bearer {self.perplexity_api_key}", "Content-Type": "application/json"}
                )

                # Raise an exception if the response status code is not 200
                response.raise_for_status()

                # Extract the summary content
                summary = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No summary generated.")
                logger.info(f"Perplexity summary generated for {row.get('Stock', 'Unknown')}.")
                return summary

            except Exception as e:
                logger.error(f"Attempt {attempt + 1}/{max_retries} - Error generating summary for {row.get('Stock', 'Unknown')}: {e}")
                if attempt < max_retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    return f"Error generating summary after {max_retries} attempts: {e}"


    async def fetch_perplexity(self, session, query, retries=3, retry_delay=2):
        """
        Fetches data from Perplexity API with retry logic.

        Args:
            session: The aiohttp session for the request.
            query: The query to send to the Perplexity API.
            retries: Number of retries in case of failure.
            retry_delay: Delay in seconds between retries.

        Returns:
            dict: The response from the Perplexity API or an error message.
        """
        api_url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.perplexity_api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [{"role": "user", "content": query}]
        }

        for attempt in range(retries):
            try:
                async with session.post(api_url, headers=headers, json=payload) as response:
                    if response.status == 200:
                        response_data = await response.json()
                        logger.info(f"Perplexity API response received for query: {query}")
                        return response_data
                    else:
                        logger.error(f"Perplexity API error: HTTP {response.status}")
                        if attempt < retries - 1:
                            logger.info(f"Retrying in {retry_delay} seconds...")
                            await asyncio.sleep(retry_delay)
                        else:
                            return {"error": f"HTTP {response.status}"}
            except Exception as e:
                logger.error(f"Error querying Perplexity API: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    await asyncio.sleep(retry_delay)
                else:
                    return {"error": str(e)}

    def fetch_perplexity_results(self, queries, retries=3, retry_delay=2):
        """
        Fetches Perplexity news results for the given queries with retry logic.

        Args:
            queries: List of queries to fetch results for.
            retries: Number of retries in case of failure.
            retry_delay: Delay in seconds between retries.

        Returns:
            list: List of responses for each query.
        """
        async def fetch_all():
            async with aiohttp.ClientSession() as session:
                tasks = [
                    self.fetch_perplexity(session, query, retries=retries, retry_delay=retry_delay)
                    for query in queries
                ]
                return await asyncio.gather(*tasks)

        return asyncio.run(fetch_all())

    async def fetch_all_news(self, queries, retries=3, retry_delay=2):
        """
        Fetches all news results using Perplexity API with retry logic.

        Args:
            queries: List of queries to fetch results for.
            retries: Number of retries in case of failure.
            retry_delay: Delay in seconds between retries.

        Returns:
            list: List of responses for each query.
        """
        async with aiohttp.ClientSession() as session:
            tasks = [
                self.fetch_perplexity(session, query, retries=retries, retry_delay=retry_delay)
                for query in queries
            ]
            return await asyncio.gather(*tasks)



    def combine_summaries(self, row):
        """
        Combine Perplexity-generated summaries into a single, coherent, and engaging news byte.
        Handles potential errors and ensures valid payloads for API requests.
        """
        try:
            # Define market hours and determine market status
            ist = timezone('Asia/Kolkata')
            current_time = datetime.now(ist)
            market_close_time = current_time.replace(hour=15, minute=30, second=0, microsecond=0)

            market_status_message = (
                "This commentary is being generated while the market is open. "
                "Let's take a recent look at today's price action and performance."
                if current_time < market_close_time else
                "This commentary is being generated after the market has closed. "
                "Here's a summary of the end-of-day performance and updates."
            )

            # Validate and clean up data for API payload
            stock = row.get('Stock', 'Unknown Stock')
            day_change = f"{row.get('Day change %', 'N/A')}%"
            contextual_summary = row.get('ContextualSummary', 'No summary available.').strip()
            perplexity_news = row.get('PerplexityNews', 'No news available.').strip()

            # Refine and truncate the summaries to fit API limitations (e.g., max characters)
            contextual_summary = contextual_summary[:1000] + "..." if len(contextual_summary) > 1000 else contextual_summary
            perplexity_news = perplexity_news[:1000] + "..." if len(perplexity_news) > 1000 else perplexity_news

            # Construct the input for Perplexity
            combined_input = (
                f"{market_status_message}\n\n"
                f"Today's top performer is {stock}, with a daily change of {day_change}.\n\n"
                f"Contextual Summary:\n{contextual_summary}\n\n"
                f"Perplexity News:\n{perplexity_news}\n\n"
                f"Generate a single, engaging, and conversational news byte. "
                f"Highlight the daily percentage change, include updates from Perplexity News, and ensure all critical details are retained. "
                f"Convert the ₹ symbol to 'rupees', round all numbers to 1 decimal place, and maintain a video-friendly tone.\n\n"
                f"Remove any text containing '**' or markdown-style formatting.\n\n"
            )

            # Use Perplexity to generate the combined summary
            payload = {
                "model": "llama-3.1-sonar-huge-128k-online",
                "messages": [{"role": "user", "content": combined_input}]
            }
            headers = {
                "Authorization": f"Bearer {self.perplexity_api_key}",
                "Content-Type": "application/json"
            }

            # API Call
            api_url = "https://api.perplexity.ai/chat/completions"
            response = requests.post(api_url, json=payload, headers=headers)

            # Handle response
            if response.status_code == 200:
                result = response.json()
                summary = result.get('choices', [{}])[0].get('message', {}).get('content', "No summary generated.")
                logger.info(f"Combined summary successfully generated for {stock}.")
                return summary.strip()

            # Log detailed error information for debugging
            else:
                logger.error(f"Error combining summaries for {stock}: {response.status_code} - {response.text}")
                return f"Error combining summaries for {stock}: {response.status_code}"

        except Exception as e:
            logger.error(f"Exception combining summaries for {row.get('Stock', 'Unknown Stock')}: {str(e)}")
            return f"Error combining summaries: {e}"



    def generate_initial_news_byte(self, df, retries=3, retry_delay=2):
        """
        Generates the initial news byte from combined summaries and daily changes with retry logic.
        """
        # Prepare the stock_changes and market_status_message dynamically
        stock_changes = "\n".join(
            f"{row['Stock']}: {row['Day change %']}%" for _, row in df.iterrows()
        )
        logger.debug(f"Stock changes prepared: {stock_changes}")

        ist = timezone("Asia/Kolkata")
        current_time = datetime.now(ist)
        market_close_time = current_time.replace(hour=15, minute=30, second=0, microsecond=0)

        market_status_message = (
            "This commentary is being generated while the market is open. "
            "Here's a quick overview of the current market performance."
            if current_time < market_close_time else
            "This commentary is being generated after the market has closed. "
            "Here's a summary of the day's top gainers and updates."
        )
        logger.debug(f"Market status message prepared: {market_status_message}")

        combined_input = (
            f"Today's top gainers and their daily percentage changes are as follows:\n\n{stock_changes}\n\n"
            f"{market_status_message}\n\n"
            + "\n".join(f"- {summary}" for summary in df['CombinedSummary'])
            + "\n\nGenerate a single, coherent, and engaging news byte that starts by summarizing the top gainers "
            "and then includes the detailed updates for each stock. Highlight any notable performance metrics such as quarterly revenue growth or EPS growth, "
            "and include any price-impacting news like earnings announcements, major deals, or market events that led to today's exceptional performance. "
            "Ensure the language is conversational, easy to understand, and all numbers and percentages are rounded to 1 decimal place. "
            "Remove any text containing '**' or markdown-style formatting.\n\n"
            "End with a reminder to stay tuned to MoneyPulse.ai for more updates.\n\n"
            "Make sure that in case a '-' is present between two numbers for instance 80-100, convert the '-' to 'to'. "
            "Convert the ₹ symbol to 'rupees'. "
            "Make sure to convert Ltd. to Limited. "
            "The entire response must be strictly limited to 2000 words or less."
        )

        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [{"role": "user", "content": combined_input}],
            "max_tokens": 1500  # Approximate token limit for 2000 words
        }

        # Retry logic
        for attempt in range(retries):
            try:
                response = requests.post(self.api_url, json=payload, headers=self.headers)
                if response.status_code == 200:
                    result = response.json()
                    news_byte = result.get('choices', [{}])[0].get('message', {}).get('content', "No news byte generated.")

                    # Clean the generated news byte
                    cleaned_news_byte = re.sub(r"\*\*.*?\*\*", "", news_byte).strip()
                    logger.info("Initial news byte generated successfully.")
                    return cleaned_news_byte
                else:
                    logger.error(f"Perplexity API error: {response.status_code} - {response.text}")
                    if attempt < retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                    else:
                        return f"Error generating initial news byte: {response.status_code}"
            except Exception as e:
                logger.error(f"Exception generating initial news byte: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    return f"Error generating initial news byte: {e}"



    def shorten_news_byte(self, initial_news_byte, retries=3, retry_delay=2):
        """
        Shortens the news byte if it exceeds 2000 words, with retry logic.
        """
        shorten_request = (
            "The previous response exceeded 2000 words. Please provide a shorter version "
            "of the following news byte, strictly under 2000 words while retaining all key information:\n\n"
            f"{initial_news_byte}\n\n"
            "Ensure the shortened version is coherent, engaging, and includes all important updates."
        )

        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [{"role": "user", "content": shorten_request}],
            "max_tokens": 1500
        }

        # Retry logic
        for attempt in range(retries):
            try:
                response = requests.post(self.api_url, json=payload, headers=self.headers)
                if response.status_code == 200:
                    result = response.json()
                    news_byte = result.get('choices', [{}])[0].get('message', {}).get('content', "No news byte generated.")

                    # Clean the shortened news byte
                    cleaned_news_byte = re.sub(r"\*\*.*?\*\*", "", news_byte).strip()
                    logger.info("Shortened news byte generated successfully.")
                    return cleaned_news_byte
                else:
                    logger.error(f"Perplexity API error: {response.status_code} - {response.text}")
                    if attempt < retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                    else:
                        return f"Error shortening news byte: {response.status_code}"
            except Exception as e:
                logger.error(f"Exception shortening news byte: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    return f"Error shortening news byte: {e}"


    def generate_final_news_byte(self, df):
        """
        Generates the final news byte, shortening if necessary.
        Ensures retries in case of errors and provides comprehensive logging.
        """
        try:
            # Generate the initial news byte
            logger.info("Starting to generate the initial news byte.")
            initial_news_byte = self.generate_initial_news_byte(df)

            if not initial_news_byte or "Error" in initial_news_byte:
                logger.error("Failed to generate the initial news byte.")
                return "Error generating the initial news byte."

            # Check the word count of the initial news byte
            word_count = len(initial_news_byte.split())
            logger.info(f"Initial news byte generated with {word_count} words.")

            # If within the word limit, return the initial news byte
            if word_count <= 1000:
                logger.info("Initial news byte is within the word limit. Returning the final news byte.")
                return initial_news_byte

            # Log and attempt to shorten the news byte
            logger.warning(f"Initial news byte exceeded 2000 words ({word_count}). Attempting to shorten.")
            retries = 3
            shortened_news_byte = None

            while retries > 0:
                try:
                    shortened_news_byte = self.shorten_news_byte(initial_news_byte)
                    final_word_count = len(shortened_news_byte.split())
                    logger.info(f"Shortened news byte generated with {final_word_count} words.")

                    # If the shortened news byte is within the limit, return it
                    if final_word_count <= 2000:
                        logger.info("Shortened news byte is within the word limit. Returning the final news byte.")
                        return shortened_news_byte
                except Exception as e:
                    retries -= 1
                    logger.error(f"Error shortening the news byte. Retries left: {retries}. Error: {e}")

            # If retries are exhausted, return the initial news byte
            logger.warning("Failed to shorten the news byte within the word limit. Returning the original news byte.")
            return initial_news_byte

        except Exception as e:
            logger.error(f"Unexpected error in generating the final news byte: {e}")
            return "Error generating the final news byte."



    def upload_image_to_s3(self, file_path, title):
        """
        Uploads an image file to an S3 bucket.
        """
        try:
            aws_access_key_id = userdata.get("aws_access_key_id")
            aws_secret_access_key = userdata.get("aws_secret_access_key")

            if not aws_access_key_id or not aws_secret_access_key:
                raise Exception("AWS credentials are not set in Colab userdata.")

            # Create an S3 client
            s3 = boto3.client(
                "s3",
                aws_access_key_id=aws_access_key_id,
                aws_secret_access_key=aws_secret_access_key,
                region_name="ap-south-1",
            )

            # Generate the S3 file key for the image file
            file_key = f"image/{title.replace(' ', '_').lower()}.png"

            # Upload the image file to the S3 bucket
            s3.upload_file(file_path, "finbytes", file_key)

            # Construct the public URL for the uploaded file
            file_url = f"https://finbytes.s3.ap-south-1.amazonaws.com/{file_key}"
            return file_url
        except Exception as e:
            raise Exception(f"Failed to upload image to S3: {str(e)}")


    def generate_plot(self, df):
        try:
            # Set the aesthetic style of the plots
            sns.set_theme(style="ticks")  # Use 'ticks' to enable control of spines

            # Define the specific green color
            green_color = ['#008000']  # Use a single green color

            # Wrap long stock names
            df['Stock'] = df['Stock'].apply(lambda x: fill(x, width=15))

            # Create the figure with mobile-friendly dimensions
            plt.figure(figsize=(6, 10))  # Adjust aspect ratio (width x height)

            # Create the horizontal bar plot
            bars = sns.barplot(
                x='Day change %',
                y='Stock',
                data=df,
                color=green_color[0],  # Single color used for the entire bar plot
                orient='h'
            )

            # Add data labels on the bars
            for bar in bars.patches:
                bars.annotate(
                    f"{bar.get_width():.1f}%",  # Format to 1 decimal place
                    (bar.get_width() + 0.2, bar.get_y() + bar.get_height() / 2),  # Position the text
                    ha='left', va='center', fontsize=10, fontweight='bold', color='white', xytext=(5, 0),
                    textcoords='offset points'
                )

            # Remove gridlines and retain only left and bottom axes
            sns.despine(left=False, bottom=False)  # Retain left and bottom borders
            plt.gca().grid(False)  # Turn off all gridlines

            # Set the main title
            plt.title('Daily % Gain Of Top Gainers', fontsize=16, fontweight='bold', pad=15, color='white')

            # Set axis labels
            plt.xlabel('Daily % Gain', fontsize=12, fontweight='bold', color='white')
            plt.ylabel('Stocks', fontsize=12, fontweight='bold', color='white')

            # Adjust font size and make stock names bold
            plt.xticks(fontsize=10, fontweight='bold', color='white')
            plt.yticks(fontsize=10, fontweight='bold', ha='right', color='white')  # Align stock names to the left

            # Set background color to black
            plt.gca().set_facecolor('black')
            plt.tight_layout()

            # Save the plot
            plt.savefig(self.plot_path, dpi=200, facecolor='black')  # Ensure the background is saved as black
            plt.close()

            logger.info(f"Mobile-friendly horizontal bar plot saved to {self.plot_path}")
        except Exception as e:
            logger.error(f"Error generating horizontal bar plot: {e}")




    def save_summary_as_json(self, final_news_byte, plot_url):
        """
        Save the final combined news byte and metadata as a JSON file.
        """
        # Get current time in IST (UTC+5:30)
        ist = timezone("Asia/Kolkata")
        created_time = datetime.now(ist).isoformat()  # Generate ISO format with timezone

        # JSON output
        output = {
            "created_time": created_time,  # Use IST with UTC+5:30 offset
            "title": "Top Gainers Today",
            "summary_text": final_news_byte,
            "image_url": plot_url,  # Include the uploaded plot URL
        }

        # Save the JSON to a file
        json_path = "/content/TopGainers_summary.json"
        try:
            with open(json_path, "w") as f:
                json.dump(output, f, indent=4)
            logger.info(f"JSON summary saved to {json_path}")
        except Exception as e:
            logger.error(f"Error saving JSON summary to {json_path}: {e}")


    def post_json_to_summary(self, json_data):
        """
        Posts a JSON summary to the specified API endpoint.

        Args:
            json_data (dict): The JSON data to be posted to the API.

        Returns:
            dict: The response from the API if successful.

        Raises:
            Exception: If the POST request fails.
        """
        api_url = "https://sy6foef31c.execute-api.ap-south-1.amazonaws.com/default/create"
        headers = {"Content-Type": "application/json"}

        try:
            response = requests.post(api_url, json=json_data, headers=headers)
            response.raise_for_status()  # Raise an error for non-2xx responses
            logger.info(f"POST request successful. Response: {response.json()}")
            return response.json()
        except Exception as e:
            logger.error(f"Error posting JSON to API: {e}")
            raise




    def run(self):
        """Main method to execute the TopGainers process."""
        logger.info("Starting the TopGainers process.")

        # Step 1: Read and preprocess the input CSV
        try:
            df = pd.read_csv(self.input_path)
        except FileNotFoundError:
            logger.error(f"Input file not found at {self.input_path}.")
            return

        # Process top 4 stocks based on "Day change %"
        if len(df) < 4:
            logger.warning("Dataset has fewer than 4 stocks. Adjusting to available rows.")
        else:
            df = df.sort_values(by="Day change %", ascending=False).head(4)

        # Step 2: Generate ChatGPT summaries
        logger.info("Generating Contextual summaries.")
        try:
            df['ContextualSummary'] = df.apply(self.generate_contextual_summary, axis=1)
        except Exception as e:
            logger.error(f"Error generating contextual summaries: {e}")
            df['ContextualSummary'] = ["Error generating summary" for _ in range(len(df))]

        # Step 3: Fetch Perplexity news
        logger.info("Fetching news from Perplexity.")
        try:
            perplexity_queries = [
                f"Find recent news and updates about {row['Stock']} in the last week." for _, row in df.iterrows()
            ]
            perplexity_results = self.fetch_perplexity_results(perplexity_queries)
            df['PerplexityNews'] = [
                res.get("choices", [{"message": {"content": "No recent news found."}}])[0]["message"]["content"]
                if isinstance(res, dict) and "error" not in res else "Error occurred"
                for res in perplexity_results
            ]
        except Exception as e:
            logger.error(f"Error fetching news from Perplexity: {e}")
            df['PerplexityNews'] = ["Error fetching news" for _ in range(len(df))]

        # Step 4: Combine ChatGPT summaries with Perplexity news
        logger.info("Combining ChatGPT summaries and Perplexity news.")
        try:
            df['CombinedSummary'] = df.apply(self.combine_summaries, axis=1)
        except Exception as e:
            logger.error(f"Error combining summaries: {e}")
            df['CombinedSummary'] = ["Error combining summaries" for _ in range(len(df))]

        # Step 5: Generate a single final news byte covering all stocks
        logger.info("Generating final news byte for all stocks.")
        try:
            final_news_byte = self.generate_final_news_byte(df)  # Pass the DataFrame to the method
            df['FinalNewsByte'] = [final_news_byte for _ in range(len(df))]  # Add the same final news byte for all rows
        except Exception as e:
            logger.error(f"Error generating final news byte: {e}")
            final_news_byte = f"Error generating final news byte: {e}"
            df['FinalNewsByte'] = [final_news_byte for _ in range(len(df))]

        # Step 6: Save the plot and upload it to S3
        logger.info("Generating and saving plot.")
        try:
            self.generate_plot(df)
            logger.info("Uploading the plot image to S3.")
            plot_url = self.upload_image_to_s3(self.plot_path, "Top Gainers Today")
            logger.info(f"Plot image successfully uploaded to S3. URL: {plot_url}")
        except Exception as e:
            logger.error(f"Error generating or uploading plot: {e}")
            plot_url = None

        # Step 7: Prepare JSON for API posting
        json_data = {
            "created_time": datetime.now(timezone('Asia/Kolkata')).isoformat(),
            "title": "Top Gainers Today",
            "summary_text": final_news_byte,
            "image_url": plot_url
        }

        # # # Step 8: Post JSON data to API
        # try:
        #     logger.info("Posting JSON data to API.")
        #     self.post_json_to_summary(json_data)
        # except Exception as e:
        #     logger.error(f"Error posting JSON data to API: {e}")

        # Step 9: Save the DataFrame to CSV
        logger.info("Saving the final output CSV.")
        try:
            df.to_csv(self.output_path, index=False)
            logger.info(f"Output saved to {self.output_path}")
        except Exception as e:
            logger.error(f"Error saving the output CSV: {e}")



# Run the class
TopGainers().run()

2024-12-19 01:44:17 - INFO - Logger initialized successfully with IST timezone.
INFO:TopGainersLogger:Logger initialized successfully with IST timezone.


File logging initialized at: /content/TopGainers_2024-12-19.log
Console logging initialized.


2024-12-19 01:44:21 - INFO - OpenAI client initialized successfully.
INFO:TopGainersLogger:OpenAI client initialized successfully.
2024-12-19 01:44:21 - INFO - TopGainers class initialized successfully.
INFO:TopGainersLogger:TopGainers class initialized successfully.
2024-12-19 01:44:21 - INFO - Input file located at: /content/TopGainers.csv
INFO:TopGainersLogger:Input file located at: /content/TopGainers.csv
2024-12-19 01:44:21 - INFO - Starting the TopGainers process.
INFO:TopGainersLogger:Starting the TopGainers process.
2024-12-19 01:44:21 - INFO - Generating Contextual summaries.
INFO:TopGainersLogger:Generating Contextual summaries.
2024-12-19 01:44:34 - INFO - Perplexity summary generated for One Mobikwik Systems Ltd..
INFO:TopGainersLogger:Perplexity summary generated for One Mobikwik Systems Ltd..
2024-12-19 01:44:49 - INFO - Perplexity summary generated for Vishal Mega Mart Ltd..
INFO:TopGainersLogger:Perplexity summary generated for Vishal Mega Mart Ltd..
2024-12-19 01:45:26

In [ ]:
# !pip install boto3
# !pip uninstall -y openai
# !pip install openai
# !pip uninstall -y httpx
# !pip install httpx